# TCGA Data Exploration
---

Exploring the preprocessed TCGA dataset from the Pancancer paper (https://www.ncbi.nlm.nih.gov/pubmed/29625048).

The Cancer Genome Atlas (TCGA), a landmark cancer genomics program, molecularly characterized over 20,000 primary cancer and matched normal samples spanning 33 cancer types. This joint effort between the National Cancer Institute and the National Human Genome Research Institute began in 2006, bringing together researchers from diverse disciplines and multiple institutions.

## Importing the necessary packages

In [ ]:
import dask.dataframe as dd                # Dask to handle big data in dataframes
import pandas as pd                        # Pandas to handle the data in dataframes
from dask.distributed import Client        # Dask scheduler
import re                                  # re to do regex searches in string data
import plotly                              # Plotly for interactive and pretty plots
import plotly.graph_objs as go
from datetime import datetime              # datetime to use proper date and time formats
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
from tqdm import tqdm_notebook             # tqdm allows to track code execution progress
import numbers                             # numbers allows to check if data is numeric
import torch                               # PyTorch to create and apply deep learning models
from torch.utils.data.sampler import SubsetRandomSampler
import utils                               # Contains auxiliary functions

Set the random seed for reproducibility:

In [ ]:
utils.set_random_seed(0)

Import the remaining custom packages:

In [ ]:
import search_explore                      # Methods to search and explore data
import data_processing                     # Data processing and dataframe operations
import embedding                           # Embedding and encoding related methods
# import padding                             # Padding and variable sequence length related methods
# import machine_learning                    # Common and generic machine learning related methods
# import deep_learning                       # Common and generic deep learning related methods

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir("../..")

# Path to the dataset files
data_path = 'storage/data/TCGA-Pancancer/'
rppa_folder = 'fcbb373e-28d4-4818-92f3-601ede3da5e1/'
dna_mthltn_folder = 'd82e2c44-89eb-43d9-b6d3-712732bf6a53/'
abs_anttd_pur_folder = '4f277128-f793-4354-a13d-30cc7fe9f6b5/'
rna_folder = '3586c0da-64d0-4b74-a449-5ff4d9136611/'
mut_folder = '1c8cfe5f-e52d-41ba-94da-f15ea1337efc/'
mirna_folder = '1c6174d9-8ffb-466e-b5ee-07b204c15cf8/'
cdr_folder = '1b5f413e-a8d1-4d10-92eb-7c4ae739ed81/'
clnc_fllw_folder = '0fc78496-818b-4896-bd83-52db1f533c5c/'
abs_anttd_seg_folder = '0f4f5701-7b61-41ae-bda9-2805d1ca9781/'

# Path to the code files
project_path = 'code/tcga-cancer-classification/'

In [ ]:
# Set up local cluster
client = Client()
client

In [ ]:
# Upload the custom methods files, so that the Dask cluster has access to relevant auxiliary functions
client.upload_file(f'{project_path}NeuralNetwork.py')
client.upload_file(f'{project_path}search_explore.py')
client.upload_file(f'{project_path}data_processing.py')
client.upload_file(f'{project_path}embedding.py')
# client.upload_file(f'{project_path}padding.py')
# client.upload_file(f'{project_path}machine_learning.py')
# client.upload_file(f'{project_path}deep_learning.py')

**Important:** Use the following two lines to be able to do plotly plots offline:

In [ ]:
import plotly.offline as py
plotly.offline.init_notebook_mode(connected=True)

## Exploring the preprocessed dataset

### RPPA data

Reverse phase protein array (RPPA) is a high-throughput antibody-based technique with the procedures similar to that of Western blots. Proteins are extracted from tumor tissue or cultured cells, denatured by SDS , printed on nitrocellulose-coated slides followed by antibody probe. Our RPPA platform currently allows for the analysis of >1000 samples using at least 130 different antibodies.

In [ ]:
rppa_df = pd.read_csv(f'{data_path}{rppa_folder}TCGA-RPPA-pancan-clean.csv')
# rppa_df = pd.read_csv(f'{data_path}TCGA-RPPA-pancan-clean.csv')
rppa_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# rppa_df = dd.read_csv(f'{data_path}{file1_folder}TCGA-RPPA-pancan-clean.csv')
# rppa_df.head()

#### Basic stats

In [ ]:
rppa_df.dtypes

In [ ]:
rppa_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(rppa_df)

Out of all the 200 columns, only 9 of them have missing values, with 8 having more than 49% (`ARID1A`, `ADAR1`, `ALPHACATENIN`, `TTF1`, `PARP1`, `JAB1`, `CASPASE9`, `CASPASE3`).

In [ ]:
rppa_df.describe().transpose()

The data is not (well) normalized yet. All columns should have 0 mean and 1 standard deviation.

#### Tumor representation

In [ ]:
rppa_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = rppa_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

This RPPA data has a **very small sample size**, specially considering how big the whole GDC/TCGA dataset really is. Furthermore, **it's significantly unbalenced**, with the most represented tumor type (BRCA) having 892 samples while the least represented tumor type (UVM) has only 12.

### RNA data

Description

In [ ]:
rna_df = pd.read_csv(f'{data_path}{rna_folder}EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', sep='\t')
# rna_df = pd.read_csv(f'{data_path}EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', sep='\t')
rna_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# rna_df = dd.read_csv(f'{data_path}{file1_folder}EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', sep='\t')
# rna_df.head()

#### Basic stats

In [ ]:
rna_df.dtypes

In [ ]:
rna_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(rna_df)

In [ ]:
rna_df.describe().transpose()

#### Tumor representation

In [ ]:
rna_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = rna_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### DNA Methylation data

Description

In [ ]:
dna_mthltn_df = pd.read_csv(f'{data_path}{dna_mthltn_folder}jhu-usc.edu_PANCAN_merged_HumanMethylation27_HumanMethylation450.betaValue_whitelisted.tsv', sep='\t')
# dna_mthltn_df = pd.read_csv(f'{data_path}jhu-usc.edu_PANCAN_merged_HumanMethylation27_HumanMethylation450.betaValue_whitelisted.tsv', sep='\t')
dna_mthltn_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# dna_mthltn_df = dd.read_csv(f'{data_path}{file1_folder}jhu-usc.edu_PANCAN_merged_HumanMethylation27_HumanMethylation450.betaValue_whitelisted.tsv', sep='\t')
# dna_mthltn_df.head()

#### Basic stats

In [ ]:
dna_mthltn_df.dtypes

In [ ]:
dna_mthltn_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(dna_mthltn_df)

In [ ]:
dna_mthltn_df.describe().transpose()

#### Tumor representation

In [ ]:
dna_mthltn_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = dna_mthltn_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### miRNA data

Description

In [ ]:
mirna_df = pd.read_csv(f'{data_path}{mirna_folder}pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv')
# mirna_df = pd.read_csv(f'{data_path}pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv')
mirna_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# mirna_df = dd.read_csv(f'{data_path}{file1_folder}pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv')
# mirna_df.head()

#### Basic stats

In [ ]:
mirna_df.dtypes

In [ ]:
mirna_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(mirna_df)

In [ ]:
mirna_df.describe().transpose()

#### Tumor representation

In [ ]:
mirna_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = mirna_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### ABSOLUTE-annotated seg data

Description

In [ ]:
abs_anttd_seg_df = pd.read_csv(f'{data_path}{abs_anttd_seg_folder}TCGA_mastercalls.abs_segtabs.fixed.txt', sep='\t')
# abs_anttd_seg_df = pd.read_csv(f'{data_path}TCGA_mastercalls.abs_segtabs.fixed.txt', sep='\t')
abs_anttd_seg_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# abs_anttd_seg_df = dd.read_csv(f'{data_path}{file1_folder}TCGA_mastercalls.abs_segtabs.fixed.txt', sep='\t')
# abs_anttd_seg_df.head()

#### Basic stats

In [ ]:
abs_anttd_seg_df.dtypes

In [ ]:
abs_anttd_seg_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(abs_anttd_seg_df)

In [ ]:
abs_anttd_seg_df.describe().transpose()

#### Tumor representation

In [ ]:
abs_anttd_seg_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = abs_anttd_seg_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### ABSOLUTE purity/ploidy data

Description

In [ ]:
abs_anttd_pur_df = pd.read_csv(f'{data_path}{abs_anttd_pur_folder}TCGA_mastercalls.abs_tables_JSedit.fixed.txt', sep='\t')
# abs_anttd_pur_df = pd.read_csv(f'{data_path}TCGA_mastercalls.abs_tables_JSedit.fixed.txt', sep='\t')
abs_anttd_pur_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# abs_anttd_pur_df = dd.read_csv(f'{data_path}{file1_folder}TCGA_mastercalls.abs_tables_JSedit.fixed.txt', sep='\t')
# abs_anttd_pur_df.head()

#### Basic stats

In [ ]:
abs_anttd_pur_df.dtypes

In [ ]:
abs_anttd_pur_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(abs_anttd_pur_df)

In [ ]:
abs_anttd_pur_df.describe().transpose()

#### Tumor representation

In [ ]:
abs_anttd_pur_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = abs_anttd_pur_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### Mutations data

Description

In [ ]:
mut_df = pd.read_csv(f'{data_path}{mut_folder}mc3.v0.2.8.PUBLIC.maf.gz', sep='\t')
# mut_df = pd.read_csv(f'{data_path}mc3.v0.2.8.PUBLIC.maf.gz', sep='\t')
mut_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# mut_df = dd.read_csv(f'{data_path}{file1_folder}mc3.v0.2.8.PUBLIC.maf.gz', sep='\t')
# mut_df.head()

#### Basic stats

In [ ]:
mut_df.dtypes

In [ ]:
mut_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(mut_df)

In [ ]:
mut_df.describe().transpose()

#### Tumor representation

In [ ]:
mut_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = mut_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### Clinical outcome (TCGA-CDR) data

Description

In [ ]:
cdr_df = pd.read_excel(f'{data_path}{cdr_folder}TCGA-CDR-SupplementalTableS1.xlsx')
# cdr_df = pd.read_excel(f'{data_path}TCGA-CDR-SupplementalTableS1.xlsx')
cdr_df.head()

In [ ]:
# For some reason Dask is failing to read the excel file directly (failed to serialize)
# cdr_df = dd.read_excel(f'{data_path}{file1_folder}TCGA-CDR-SupplementalTableS1.xlsx')
# cdr_df.head()

#### Basic stats

In [ ]:
cdr_df.dtypes

In [ ]:
cdr_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(cdr_df)

In [ ]:
cdr_df.describe().transpose()

#### Tumor representation

In [ ]:
cdr_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = cdr_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)

### Clinical with follow-up data

Description

In [ ]:
clnc_fllw_df = pd.read_csv(f'{data_path}{clnc_fllw_folderclinical_PANCAN_patient_with_followup.tsv', sep='\t')
# clnc_fllw_df = pd.read_csv(f'{data_path}clinical_PANCAN_patient_with_followup.tsv', sep='\t')
clnc_fllw_df.head()

In [ ]:
# For some reason Dask is failing to read the CSV file directly (failed to serialize)
# clnc_fllw_df = dd.read_csv(f'{data_path}{file1_folder}clinical_PANCAN_patient_with_followup.tsv', sep='\t')
# clnc_fllw_df.head()

#### Basic stats

In [ ]:
clnc_fllw_df.dtypes

In [ ]:
clnc_fllw_df.nunique()

In [ ]:
search_explore.dataframe_missing_values(clnc_fllw_df)

In [ ]:
clnc_fllw_df.describe().transpose()

#### Tumor representation

In [ ]:
clnc_fllw_df.TumorType.value_counts().to_frame()

In [ ]:
data = [go.Histogram(x = clnc_fllw_df.TumorType)]
layout = go.Layout(title='Tumor types representation',
                   plot_bgcolor='#ffffff',
                   xaxis=dict(categoryorder='total descending'))
fig = go.Figure(data, layout)
py.iplot(fig)